In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

In [4]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [5]:
pd.options.display.max_colwidth = 100

In [6]:
mass = pd.read_pickle('../data/01_EDA_participant_numbers.pkl')

In [7]:
mass.head()

,id,country,ccode,year,region,protestnumber,startday,startmonth,startyear,endday,...,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,protester_id_type,partipants_number,protest_size_category
0,201990001,Canada,20,1990,North America,1,15.0,1.0,1990.0,15.0,...,NaN,NaN,NaN,NaN,NaN,"1. great canadian train journeys into history; passenger rail system cut the times london , janu...","canada s railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by...",workers_unions,1000,"1,000-4,999"
1,201990002,Canada,20,1990,North America,2,25.0,6.0,1990.0,25.0,...,NaN,NaN,NaN,NaN,NaN,"1. autonomy s cry revived in quebec the new york times, june 25, 1990, monday, late edition fina...","protestors were only identified as young people, and the gathering was spontaneous",students_youth,1000,"1,000-4,999"
2,201990003,Canada,20,1990,North America,3,1.0,7.0,1990.0,1.0,...,NaN,NaN,NaN,NaN,NaN,"1. quebec protest after queen calls for unity the times, july 2, 1990, monday, overseas news, 70...","the queen, after calling on canadians to remain united, braved protesters and entered french spe...",political_group,500,100-999
3,201990004,Canada,20,1990,North America,4,12.0,7.0,1990.0,6.0,...,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed confrontation in canada reflects growing militancy...,canada s federal government has agreed to acquire a tract of land that has been the subject of a...,ethnic_group,100,100-999
4,201990005,Canada,20,1990,North America,5,14.0,8.0,1990.0,15.0,...,accomodation,NaN,NaN,NaN,NaN,"1. dozens hurt in mohawk blockade protest the times, august 14, 1990, tuesday, overseas news, 25...",protests were directed against the state due to its refusal to use violence to clear mohawk indi...,locals_residents,950,100-999


In [8]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15227 entries, 0 to 17141
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     15227 non-null  int64  
 1   country                15227 non-null  object 
 2   ccode                  15227 non-null  int64  
 3   year                   15227 non-null  int64  
 4   region                 15227 non-null  object 
 5   protestnumber          15227 non-null  int64  
 6   startday               15227 non-null  float64
 7   startmonth             15227 non-null  float64
 8   startyear              15227 non-null  float64
 9   endday                 15227 non-null  float64
 10  endmonth               15227 non-null  float64
 11  endyear                15227 non-null  float64
 12  protesterviolence      15227 non-null  float64
 13  location               15206 non-null  object 
 14  protesteridentity      15227 non-null  object 
 15  pr

In [9]:
sorted(list(mass['country'].value_counts().index))[:10]

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh']

In [10]:
mass['region'].value_counts()

Europe             4992
Africa             3181
Asia               3126
South America      1659
MENA               1259
North America       527
Central America     445
Oceania              38
Name: region, dtype: int64

In [11]:
# mass[mass['region']== 'Central America'].groupby('country').count()

In [12]:
# mass[mass['region']== 'North America'].groupby('country').count()

In [13]:
mass['notes_clean'] = mass['notes']

In [14]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'protesteridentity',
       'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
       'protesterdemand4', 'stateresponse1', 'stateresponse2',
       'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6',
       'stateresponse7', 'sources', 'notes', 'protester_id_type',
       'partipants_number', 'protest_size_category', 'notes_clean'],
      dtype='object')

In [15]:
def standardize_text(df, text_field):
    
    '''Removes special characters and hyperlinks from dataframe'''
    
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"'", "")
    df[text_field] = df[text_field].str.replace(r"\s+[a-zA-Z]\s+", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    
    return df

In [16]:
standardize_text(mass, 'notes_clean')

,id,country,ccode,year,region,protestnumber,startday,startmonth,startyear,endday,...,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,protester_id_type,partipants_number,protest_size_category,notes_clean
0,201990001,Canada,20,1990,North America,1,15.0,1.0,1990.0,15.0,...,NaN,NaN,NaN,NaN,"1. great canadian train journeys into history; passenger rail system cut the times london , janu...","canada s railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by...",workers_unions,1000,"1,000-4,999","canada railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by v..."
1,201990002,Canada,20,1990,North America,2,25.0,6.0,1990.0,25.0,...,NaN,NaN,NaN,NaN,"1. autonomy s cry revived in quebec the new york times, june 25, 1990, monday, late edition fina...","protestors were only identified as young people, and the gathering was spontaneous",students_youth,1000,"1,000-4,999","protestors were only identified as young people, and the gathering was spontaneous"
2,201990003,Canada,20,1990,North America,3,1.0,7.0,1990.0,1.0,...,NaN,NaN,NaN,NaN,"1. quebec protest after queen calls for unity the times, july 2, 1990, monday, overseas news, 70...","the queen, after calling on canadians to remain united, braved protesters and entered french spe...",political_group,500,100-999,"the queen, after calling on canadians to remain united, braved protesters and entered french spe..."
3,201990004,Canada,20,1990,North America,4,12.0,7.0,1990.0,6.0,...,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed confrontation in canada reflects growing militancy...,canada s federal government has agreed to acquire a tract of land that has been the subject of a...,ethnic_group,100,100-999,canada federal government has agreed to acquire tract of land that has been the subject of armed...
4,201990005,Canada,20,1990,North America,5,14.0,8.0,1990.0,15.0,...,NaN,NaN,NaN,NaN,"1. dozens hurt in mohawk blockade protest the times, august 14, 1990, tuesday, overseas news, 25...",protests were directed against the state due to its refusal to use violence to clear mohawk indi...,locals_residents,950,100-999,protests were directed against the state due to its refusal to use violence to clear mohawk indi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17136,9102014001,Papua New Guinea,910,2014,Oceania,1,16.0,2.0,2014.0,18.0,...,NaN,NaN,NaN,NaN,"probe into killing of manus detainee; manus island death png violence one dead, 77 injured, coun...",? a government inquiry will be launched as serious questions have been raised about australia s ...,locals_residents,100,100-999,? government inquiry will be launched as serious questions have been raised about australia resp...
17138,9102016001,Papua New Guinea,910,2016,Oceania,1,15.0,5.0,2016.0,9.0,...,NaN,NaN,NaN,NaN,papua new guinea: reports of up to four people killed in police shooting at student protest; pro...,police in papua new guinea fired gunshots wednesday to quell a student protest demanding the pri...,students_youth,1000,"1,000-4,999",police in papua new guinea fired gunshots wednesday to quell student protest demanding the prime...
17139,9102017001,Papua New Guinea,910,2017,Oceania,1,15.0,6.0,2017.0,15.0,...,NaN,NaN,NaN,NaN,bougainville imposes moratorium on panguna mine over fears of civil unrest; in dramatic policy t...,the bougainville government has enacted an indefinite moratorium on renewing the licence of a co...,political_group,50,50-99,the bougainville government has enacted an indefinite moratorium on renewing the licence of cont...
17140,9102017002,Papua New Guinea,910,2017,Oceania,2,15.0,7.0,2017.0,15.0,...,NaN,NaN,NaN,NaN,"violence, chaos and fraud: fraught papua new guinea election returns prime minister peter o neil...",peter o neill has been reappointed as prime minister of papua new guinea amid a shaky start to t...,political_group,50,50-99,peter neill has been reappointed as pr

In [42]:
pd.options.display.max_colwidth = 300

In [17]:
mass[ mass['notes'] != mass['notes_clean'] ]

,id,country,ccode,year,region,protestnumber,startday,startmonth,startyear,endday,...,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,protester_id_type,partipants_number,protest_size_category,notes_clean
0,201990001,Canada,20,1990,North America,1,15.0,1.0,1990.0,15.0,...,NaN,NaN,NaN,NaN,"1. great canadian train journeys into history; passenger rail system cut the times london , janu...","canada s railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by...",workers_unions,1000,"1,000-4,999","canada railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by v..."
2,201990003,Canada,20,1990,North America,3,1.0,7.0,1990.0,1.0,...,NaN,NaN,NaN,NaN,"1. quebec protest after queen calls for unity the times, july 2, 1990, monday, overseas news, 70...","the queen, after calling on canadians to remain united, braved protesters and entered french spe...",political_group,500,100-999,"the queen, after calling on canadians to remain united, braved protesters and entered french spe..."
3,201990004,Canada,20,1990,North America,4,12.0,7.0,1990.0,6.0,...,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed confrontation in canada reflects growing militancy...,canada s federal government has agreed to acquire a tract of land that has been the subject of a...,ethnic_group,100,100-999,canada federal government has agreed to acquire tract of land that has been the subject of armed...
4,201990005,Canada,20,1990,North America,5,14.0,8.0,1990.0,15.0,...,NaN,NaN,NaN,NaN,"1. dozens hurt in mohawk blockade protest the times, august 14, 1990, tuesday, overseas news, 25...",protests were directed against the state due to its refusal to use violence to clear mohawk indi...,locals_residents,950,100-999,protests were directed against the state due to its refusal to use violence to clear mohawk indi...
5,201990006,Canada,20,1990,North America,6,19.0,9.0,1990.0,19.0,...,NaN,NaN,NaN,NaN,"1. mohawks tear gassed in clash the washington post, september 19, 1990, wednesday, final editio...",canadian army troops today fired tear gas into a crowd of about 200 mohawk indians who attempted...,ethnic_group,200,100-999,canadian army troops today fired tear gas into crowd of about 200 mohawk indians who attempted t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17136,9102014001,Papua New Guinea,910,2014,Oceania,1,16.0,2.0,2014.0,18.0,...,NaN,NaN,NaN,NaN,"probe into killing of manus detainee; manus island death png violence one dead, 77 injured, coun...",? a government inquiry will be launched as serious questions have been raised about australia s ...,locals_residents,100,100-999,? government inquiry will be launched as serious questions have been raised about australia resp...
17138,9102016001,Papua New Guinea,910,2016,Oceania,1,15.0,5.0,2016.0,9.0,...,NaN,NaN,NaN,NaN,papua new guinea: reports of up to four people killed in police shooting at student protest; pro...,police in papua new guinea fired gunshots wednesday to quell a student protest demanding the pri...,students_youth,1000,"1,000-4,999",police in papua new guinea fired gunshots wednesday to quell student protest demanding the prime...
17139,9102017001,Papua New Guinea,910,2017,Oceania,1,15.0,6.0,2017.0,15.0,...,NaN,NaN,NaN,NaN,bougainville imposes moratorium on panguna mine over fears of civil unrest; in dramatic policy t...,the bougainville government has enacted an indefinite moratorium on renewing the licence of a co...,political_group,50,50-99,the bougainville government has enacted an indefinite moratorium on renewing the licence of cont...
17140,9102017002,Papua New Guinea,910,2017,Oceania,2,15.0,7.0,2017.0,15.0,...,NaN,NaN,NaN,NaN,"violence, chaos and fraud: fraught papua new guinea election returns prime minister peter o neil...",peter o neill has been reappointed as prime minister of papua new guinea amid a shaky start to t...,political_group,50,50-99,peter neill has b

In [18]:
mass[['notes', 'notes_clean']]

,notes,notes_clean
0,"canada s railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by...","canada railway passenger system was finally cut by half yesterday, with 18 of 38 routes run by v..."
1,"protestors were only identified as young people, and the gathering was spontaneous","protestors were only identified as young people, and the gathering was spontaneous"
2,"the queen, after calling on canadians to remain united, braved protesters and entered french spe...","the queen, after calling on canadians to remain united, braved protesters and entered french spe..."
3,canada s federal government has agreed to acquire a tract of land that has been the subject of a...,canada federal government has agreed to acquire tract of land that has been the subject of armed...
4,protests were directed against the state due to its refusal to use violence to clear mohawk indi...,protests were directed against the state due to its refusal to use violence to clear mohawk indi...
...,...,...
17136,? a government inquiry will be launched as serious questions have been raised about australia s ...,? government inquiry will be launched as serious questions have been raised about australia resp...
17138,police in papua new guinea fired gunshots wednesday to quell a student protest demanding the pri...,police in papua new guinea fired gunshots wednesday to quell student protest demanding the prime...
17139,the bougainville government has enacted an indefinite moratorium on renewing the licence of a co...,the bougainville government has enacted an indefinite moratorium on renewing the licence of cont...
17140,peter o neill has been reappointed as prime minister of papua new guinea amid a shaky start to t...,peter neill has been reappointed as prime minister of papua new guinea amid shaky start to the n...
